# Simulation to extend Hanna & Olken (2018)
## Universal Basic Incomes versus Targeted Transfers: Anti-Poverty Programs in Developing Countries

Consider different budget levels, and a mix of UBI and targeted transfers.

Simulation notebook.

## Setup

In [1]:
def import_or_install(package, pip_install=None):
    """ Try to install a package, and pip install if it's unavailable.
    
    Args:
        package: Package name.
        pip_install: Location to pip install from.
            Runs `pip install [package]` if not provided.
    """
    import pip
    if pip_install is None:
        pip_install = package
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])

In [2]:
import_or_install('pandarallel')

In [3]:
import pandas as pd
import numpy as np
import os
import microdf as mdf

In [4]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Load data

[This notebook](https://colab.research.google.com/drive/1dxg8kjXHV7Fc-qKlaA0LjNPFrzLD0JVM) downloads this file directly from the Census Bureau.

In [5]:
SPM_COLS = ['SPM_ID', 'SPM_NUMPER', 'SPM_RESOURCES', 'SPM_POVTHRESHOLD',
            'SPM_WEIGHT']

In [6]:
raw = pd.read_csv(
    'https://github.com/MaxGhenis/datarepo/raw/master/pppub19.csv.gz',
    usecols=SPM_COLS + ['MARSUPWT'])

Source: [World Bank](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=US) (as of 2018)

In [7]:
US_GDP = 20.5e12

## Preprocess

In [8]:
u = raw.groupby(SPM_COLS).sum()
u.reset_index([i for i in SPM_COLS if i != 'SPM_ID'], inplace=True)

Define `y` to be resources per person. Set values below \$1 to \$1 so that CRRA works.

In [9]:
u['y0'] = np.maximum(1., u.SPM_RESOURCES / u.SPM_NUMPER)

In [10]:
u['w'] = u.SPM_WEIGHT / 100

Assign weighted rank by income.

In [11]:
u.sort_values('y0', inplace=True)
u['y0_rank'] = u.w.cumsum()
u['y0_pr'] = u.y0_rank / u.w.sum()

### Add noisy income

The actual value of the noisy income isn't important, since it's only used for ranking households. Therefore, random normal noise is sufficient.

Set noise level to match Hanna and Olken's model:
> The typical fit we found of these regressions (the R2) is between 0.53 and 0.66

Their [appendix](https://www.aeaweb.org/content/file?id=8344) shows that they were predicting log income.

Shoot for average: 0.595.

In [12]:
np.random.seed(0)

In [13]:
TARGET_R2 = 0.595

In [14]:
def log_noise(y, noise_mean):
    return np.exp(np.log(y) + noise_mean * np.random.randn(len(y)))

In [15]:
def r2(noise_mean):
    y_noise = log_noise(u.y0, noise_mean)
    r = np.corrcoef(np.log(u.y0), np.log(y_noise))[0, 1]
    return np.power(r, 2)

In [16]:
NOISE_LEVEL = 1.37
r2(NOISE_LEVEL)  # Close to 0.595.

0.5981376687619906

In [17]:
r2(NOISE_LEVEL * 2)

0.2709802289873735

In [18]:
u['y0_l_noise'] = log_noise(u.y0, NOISE_LEVEL)
u['y0_h_noise'] = log_noise(u.y0, NOISE_LEVEL * 2)

Re-rank.

In [19]:
u.sort_values('y0_l_noise', inplace=True)
u['y0_rank_l_noise'] = u.w.cumsum()
u['y0_pr_l_noise'] = u.y0_rank_l_noise / u.w.sum()

In [20]:
u.sort_values('y0_h_noise', inplace=True)
u['y0_rank_h_noise'] = u.w.cumsum()
u['y0_pr_h_noise'] = u.y0_rank_h_noise / u.w.sum()

Check R-squared from noisy to true income rank.

**Low noise**

In [21]:
u[['y0_rank', 'y0_rank_l_noise']].corr().iloc[0, 1]

0.5128437390086074

**High noise**

In [22]:
u[['y0_rank', 'y0_rank_h_noise']].corr().iloc[0, 1]

0.3205836508134436

## Analysis

### Define CRRA function

In [23]:
def crra(y, w=None, rho=3):
    """ Constant relative risk-aversion social welfare function.

    Args:
        y: Array of after-tax after-transfer income.
        w: Optional array of weights. Should be the same length as y.
        rho: Coefficient of relative risk-aversion, where higher values of rho
             put higher weights on transfers received by the very poor.
             Defaults to 3 per Hanna and Olken (2018).
        
    Returns:
        CRRA SWF. Also sets any value below 1 to 1.
    """
    num = np.power(np.array(y, dtype=float), 1 - rho)
    if w is not None:
        num *= w
    return num.sum() / (1 - rho)

Status quo CRRA value.

In [24]:
crra0 = crra(u.y0, u.w)
crra0

-1542708.9725729444

### Define horizontal equity function

From Hanna and Olken (2018):
>At each cutoff c, we calculate, for each household, the percentage of households within ±5 income percentiles (based on actual income) that received the same benefit status—included or excluded—based on the results of proxy-means test prediction. In other words, for households that were included in the program at a given c, we calculate the percentage of similar households that were also included; for households that were excluded, we calculate the percentage of similar households that were also excluded.

**TODO**

### Define simulation function

In [25]:
total_hhs = u.w.sum()  # Number of SPM units.

In [26]:
def simulate(budget_share_of_gdp, pr_threshold, ubi_share, income_pr_col):
    """ Simulate a transfer split between targeted and UBI components.

    Args:
        budget_share_of_gdp: Total budget to be split between targeted and UBI
                             components, as a share of US GDP (0 to 100).
        pr_threshold: Percentrank below which households get the targeted
                      transfer. 0 to 100.
        ubi_share: Number between 0 and 100 representing the share of the
                   transfer that goes to a UBI.
        income_col: Column indicating the income percent rank (true or noisy).
    
    Returns:
    """
    budget = US_GDP * budget_share_of_gdp / 100
    ubi_budget = budget * (ubi_share / 100)
    targeted_budget = budget * (1 - ubi_share / 100)
    ubi_amount = ubi_budget / total_hhs
    target_idx = u[income_pr_col] < (pr_threshold / 100)
    target_hhs = u[target_idx].w.sum()
    targeted_amount = targeted_budget / target_hhs
    return u.y0 + ubi_amount + np.where(target_idx, targeted_amount, 0)

## Simulate

Cartesian product function from https://github.com/MaxGhenis/microdf/blob/master/microdf/utils.py

In [27]:
SIMX = {
    'budget_share_of_gdp': [0.01, 0.1, 0.2, 0.5, 1, 5],
    'noise_col': ['y0_pr', 'y0_pr_l_noise', 'y0_pr_h_noise'],
    'pr_threshold': np.arange(0, 101, 1),
    'ubi_share': np.arange(0, 101, 1)
    }
sim = mdf.cartesian_product(SIMX)

Usually takes 25 minutes.

In [28]:
%%time
sim['crra'] = sim.parallel_apply(
    lambda row: crra(simulate(row.budget_share_of_gdp, row.pr_threshold,
                              row.ubi_share, row.noise_col)),
    axis=1)

/home/mghenis/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/mghenis/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/mghenis/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/home/mghenis/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/home/mghenis/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/mghenis/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/home/mghenis/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/home/mghenis/miniconda

CPU times: user 1.72 s, sys: 808 ms, total: 2.53 s
Wall time: 24min 21s


## Postprocess
Make the noise column a category.

In [29]:
sim['noise'] = pd.Categorical(
    np.where(sim.noise_col == 'y0_pr', 'No noise',
             np.where(sim.noise_col == 'y0_pr_l_noise', 'Low noise',
                      'High noise')),
                      categories = ['No noise', 'Low noise', 'High noise'])

In [30]:
sim.drop(['noise_col'], axis=1, inplace=True)

## Export

In [31]:
sim.to_csv('sim.csv', index=False)